In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!apt-get install sox libsndfile1 ffmpeg
!pip install wget
!pip install text-unidecode

# ### Install ATOMMIC
BRANCH = 'main'
!python -m pip install git@github.com:wdika/atommic.git

In [ ]:
### Install Hugging Face Hub
!python -m pip install huggingface_hub
!python -m pip install evaluate

# ATOMMIC models on Hugging Face Hub

This guide will briefly show how to upload ATOMMIC models to Hugging Face programmatically.

This enables community members to share their ATOMMIC models (any model!) with all users of ATOMMIC!

**Note**: While in this tutorial we showcase a reconstruction model, there is no particular restriction to any domain - all ATOMMIC models (.atommic files) of every domain can be uploaded and shared in the same way.

# Login to Hugging Face

Use the notebook login, and access your user access token (or create one to upload models to Hugging Face).

For more information, visit the User Access Token section - https://huggingface.co/docs/hub/security-tokens

In [ ]:
from huggingface_hub import notebook_login

# allow to enter token manually if not in notebook
notebook_login()

In [ ]:
!git config --global credential.helper store

# Prepare a model to upload to HF

In this example, we will upload an ATOMMIC REC model to Hugging Face for simplicity and to showcase the method.

**You can swap out this REC model for any model that you restore via `restore_from()` and follow the same steps to upload your own models !**

In [ ]:
from omegaconf import OmegaConf, open_dict

In [ ]:
import atommic.collections.reconstruction as atommic_rec

# Model Name

ATOMMIC adheres to strict requirements when naming a model for upload to Hugging Face Hub. 

It is **mandatory** to share the model name across the model card, the ATOMMIC file itself. Otherwise ATOMMIC model from Hugging Face will fail to restore correctly.

## Naming Convention

ATOMMIC model names can vary based on the task. Following the standard guidelines when naming models, we do not expect the same level of strictness for community contributions.

Here are some common guidelines we encourage (but do not enforce) users to follow : 

- `Task name`: Usually a short 3-4 character representation of the task that the model performs.
  - `mtl`  = MultiTask Learning (MTL)
  - `qmri` = quantitative MRI (qMRI)
  - `rec`  = Reconstruction (REC)
  - `seg`  = Segmentation (SEG)

- `Model Identifier`: Since models vary so drastically across domains, there is a lot of flexibility here. We try to adhere to naming conventions in literature as much as possible. For example, you can attach `model architecture` (REC/UNet), `training loss` (REC/SSIM), and `model size` (small, large, etc.).

- `Optional: Additional Modifiers`: These are additional identifiers such as dataset name (cc359 for Calgary-Campinas 359), etc. It can be set on a case-by-case basis.

All these name segments are jointed by `_`.

-----

As an example of the following model we will try today : 

`{task name}_{model identifier}_[OPTIONAL modifiers]` = `rec_unet_small`

**Set the MODEL_NAME carefully** !

In [ ]:
MODEL_NAME = "REC_UNet_CC359_12_channel_poisson2d_5x_10x_NNEstimationCSM"

-----
**Restore a ATOMMIC Model**

Here, we restore a model from a local .atommic file using `restore_from()`.

In [ ]:
path_to_local_model = input("Please enter the (local) path to the pre-trained model file : ")

In [ ]:
model, _ = atommic_rec.nn.UNet.restore_from(path_to_local_model)

# Create a Hugging Face Model

Now that we have an ATOMMIC model and have logged into Hugging Face with our user API key, we can begin by creating a new repository and uploading our model.

-----

After the model has been restored, create an HfApi object to interact with the model repository.

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
username = api.whoami()['name']

In [ ]:
try:
  api.create_repo(repo_id=MODEL_NAME)
  print("Successfully created repository !")
except Exception as e:
  print("Repository is possibly already created. Refer to error here - \n\n", e)

In [ ]:
from huggingface_hub import Repository

Note two essential names - 

- `hf_model_name`: A string name that is the composite of your `username` and `MODEL_NAME` as set above. This name is used for multiple purposes, so keep track of it.

- `model_filename`: The actual filename of the ATOMMIC model that will be uploaded to Hugging Face. Note that this filename is explicitly set to `{MODEL_NAME}.atommic`. If this model filename is altered, then the model cannot correctly be restored by ATOMMIC when downloaded from Hugging Face Hub, so please be careful.

In [ ]:
local_dir = f'model-{MODEL_NAME}/'
hf_model_name = f'{username}/{MODEL_NAME}'

commit_message = "Upload model"
model_filename = f'{MODEL_NAME}.atommic'

with Repository(local_dir=local_dir, clone_from=hf_model_name, repo_type='model').commit(commit_message):
  model.save_to(model_filename)

In [ ]:
print("Finished uploading model to :", hf_model_name)

## Test if the model works 

Now that we uploaded the model, let's try to use it in ATOMMIC !

The only change required between normally calling `from_pretrained(model_name)` is to call **`from_pretrained({username}/{filename})`**

In [ ]:
hf_model_name = f'{username}/{MODEL_NAME}'
hf_model, _ = atommic_rec.nn.UNet.from_pretrained(hf_model_name)

In [ ]:
print("Successfully used HF model -", hf_model_name)

# Model Card

Now that we have uploaded the model, we are nearly 50% done!

The next step is to update the model card to have some helpful information regarding the uploaded model and its scores compared to other models.

You can do this in two ways, manually (by clicking the link below) or programmatically fill in part of the model card by following the instructions below.

In [ ]:
hf_url = f'https://huggingface.co/{username}/{MODEL_NAME}'
print(f"Visit {hf_url} to manually edit your model card")

-----

Here, we are going to setup some variables for our model card.

First up are the tags:

In [ ]:
TAGS = [
    "image-reconstruction",
    "UNet",
    "ATOMMIC",  # required for library identification
    "pytorch",  # required, for toolkit identification
]

-----

Next, we list down all the datasets that were used to train the model.

By convention, try to search if the dataset already exists on Hugging Face Datasets - it is usually listed at the top and in lower case.

If you train on datasets that don't yet exist in Hugging Face Datasets, you can still add them but try to differentiate them by using capitalized names.

In [ ]:
# Replace all spaces with `-`
DATASETS = [
    "CC359",
]

-----
Now we create an automated template based on a config for the top portion of the readme file.

In [ ]:
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any

@dataclass
class ATOMMICHuggingFaceModelConfig:
  language: List[str]
  license: str

  library_name: str = "atommic"
  datasets: List[str] = field(default_factory=lambda: DATASETS)
  thumbnail: Optional[str] = None
  tags: List[str] = field(default_factory=lambda: TAGS)
  model_index: Any = field(default_factory=lambda: [dict(name=MODEL_NAME, results=[])])

In [ ]:
config = ATOMMICHuggingFaceModelConfig(language=['en'], license="cc-by-4.0")  # choose appropriate license here
config = OmegaConf.structured(config)

with open_dict(config):
  # Update `model_index` to `model-index`
  model_index = config.pop('model_index')
  config['model-index'] = model_index

  # Replace all spaces with `-` in datasets
  normalized_datasets = [ds_name.replace(" ", "-") for ds_name in config['datasets']]
  config['datasets'] = OmegaConf.create(normalized_datasets)

print(OmegaConf.to_yaml(config))

## Markdown Template

Now that we have an auto-generated header for our readme, next, we write down some template markdown for the actual contents of the markdown.

You can edit the code here directly if you want, or if you prefer the GUI to see the actual changes in real-time, you can finish uploading this model card and then edit the readme file on the Hugging Face webpage itself.

In [ ]:
hf_model_name = f'{username}/{MODEL_NAME}'

TEMPLATE = f"""
## Model Overview

UNet for 5x & 10x accelerated MRI Reconstruction on the CC359 dataset


## ATOMMIC: Training

To train, fine-tune or play with the model you will need to install [ATOMMIC](https://github.com/wdika/atommic). We recommend you install it after you've installed latest Pytorch version.
```
pip install atommic['all']
``` 

## How to Use this Model

The model is available for use in ATOMMIC, and can be used as a pre-trained checkpoint for inference or for fine-tuning on another dataset.

Corresponding configuration YAML files can be found [here](https://github.com/wdika/atommic/tree/main/projects/REC/CC359/conf).

### Automatically instantiate the model

```python
import atommic.collections.reconstruction.nn as atommic_rec_nn
atommic_rec_model = atommic_rec_nn.unet.UNet.from_pretrained("{hf_model_name}")
```

### Usage

You need to download the CC359 dataset to effectively use this model. Check the [CC359](https://github.com/wdika/atommic/blob/main/projects/REC/CC359/README.md) page for more information.


## Model Architecture
```base
model_name: UNet
channels: 64
pooling_layers: 4
in_channels: 2
out_channels: 2
padding_size: 11
dropout: 0.0
normalize: true
norm_groups: 2
dimensionality: 2
reconstruction_loss:
    l1: 0.1
    ssim: 0.9
```

## Training
```base
optim:
    name: adamw
    lr: 1e-4
    betas:
        - 0.9
        - 0.999
    weight_decay: 0.0
    sched:
        name: CosineAnnealing
        min_lr: 0.0
        last_epoch: -1
        warmup_ratio: 0.1

trainer:
  strategy: ddp_find_unused_parameters_false
  accelerator: gpu
  devices: 1
  num_nodes: 1
  max_epochs: 50
  precision: 16-mixed
  enable_checkpointing: false
  logger: false
  log_every_n_steps: 50
  check_val_every_n_epoch: -1
  max_steps: -1
```

## Performance

To compute the targets using the raw k-space and the chosen coil combination method, accompanied with the chosen coil sensitivity maps estimation method, you can use [targets](https://github.com/wdika/atommic/tree/main/projects/REC/CC359/conf/targets) configuration files.

Evaluation can be performed using the [evaluation](https://github.com/wdika/atommic/blob/main/tools/evaluation/reconstruction.py) script for the reconstruction task.

Results
-------

Evaluation against RSS targets
------------------------------
5x: MSE = 0.001429 +/- 0.001373 NMSE = 0.02208 +/- 0.02319 PSNR = 28.85 +/- 4.169 SSIM = 0.8487 +/- 0.07037

10x: MSE = 0.002108 +/- 0.002 NMSE = 0.03273 +/- 0.03417 PSNR = 27.2 +/- 4.197 SSIM = 0.8095 +/- 0.09149


## Limitations

This model was trained on the CC359 using automatic coil sensitivity maps estimation and might differ from the results reported on the challenge leaderboard.


## References

[1] [ATOMMIC](https://github.com/wdika/atommic)

[2] Beauferris, Y., Teuwen, J., Karkalousos, D., Moriakov, N., Caan, M., Yiasemis, G., Rodrigues, L., Lopes, A., Pedrini, H., Rittner, L., Dannecker, M., Studenyak, V., Gröger, F., Vyas, D., Faghih-Roohi, S., Kumar Jethi, A., Chandra Raju, J., Sivaprakasam, M., Lasby, M., … Souza, R. (2022). Multi-Coil MRI Reconstruction Challenge—Assessing Brain MRI Reconstruction Models and Their Generalizability to Varying Coil Configurations. Frontiers in Neuroscience, 16. https://doi.org/10.3389/fnins.2022.919186
"""

-----

Below, we will upload this model card in a temporary file called **`"readme_template.md"`**. This is done to prevent overwriting of the "final" model card that the user may have manually edited.

Once this step is finished, **please copy the contents of this file, create a README.md file and paste the contents into it**.

In [ ]:
local_dir = f'model-{MODEL_NAME}/'
hf_model_name = f'{username}/{MODEL_NAME}'

commit_message = "Upload config"
filename = 'readme_template.md'

with Repository(local_dir=local_dir, clone_from=hf_model_name, repo_type='model').commit(commit_message):
  with open(filename, 'w') as f:
    f.write("---\n")
    f.write(OmegaConf.to_yaml(config))
    f.write("\n---\n\n")
    f.write(TEMPLATE)
  

-----

Please visit the URL below to copy the contents of the `readme_template.md` file into your `README.md` file.

In [ ]:
hf_url = f'https://huggingface.co/{username}/{MODEL_NAME}'
print(f"Visit {hf_url} to edit your model card from the generated template file `{filename}`")

## Evaluation Results

Now that we have both the model checkpoint and the readme uploaded to the Hub, we can optionally add some evaluation results to the card as well!

However, HF doesn't support (yet) the image-reconstruction task and logging metrics is not possible. You can log metrics for segmentation, if logging a segmentation model.

In [ ]:
!pip install cchardet

In [ ]:
import evaluate

In [ ]:
hf_model_name = f'{username}/{MODEL_NAME}'

# evaluate.push_to_hub(
#     model_id=hf_model_name,
#     task_type="segmentation",
#     dataset_type="",
#     dataset_name="",
#     metric_type="",
#     metric_name="",
#     dataset_split="",
#     dataset_config="",
#     metric_value=1.0,
# )

-----

Done! Now we have a model checkpoint, a model card as well as evaluation results all set up for the ATOMMIC model on Hugging Face!

To add more metrics, you can copy-paste the above cell and repeat the procedure for as many metrics as needed!

### Optionally you might want to remove any generated dirs

In [ ]:
import os

In [ ]:
# remove dir with MODEL_NAME in tutorials dir
os.system(f"rm -rf tutorials/{MODEL_NAME}")
# remove .ipynb checkpoints
os.system(f"rm -rf tutorials/.ipynb_checkpoints")